<a href="https://colab.research.google.com/github/vfrantc/TransMod/blob/main/processing_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm
!pip install thop
!pip install kornia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 5.0 MB/s 


In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


# Run Transweather

In [3]:
!git clone https://github.com/vfrantc/TransMod.git
%cd TransMod
!mkdir -p data/test
!cp /content/drive/MyDrive/deweather/input.zip .
!unzip input.zip
!cp -r input data/test/
!cp input/natural.txt data/test/input.txt
!mkdir -p data/test/gt
!cp -r input/input data/test/gt/gt

Cloning into 'TransMod'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 420 (delta 101), reused 87 (delta 50), pack-reused 264
Receiving objects: 100% (420/420), 11.12 MiB | 24.71 MiB/s, done.
Resolving deltas: 100% (222/222), done.
/content/TransMod
Archive:  input.zip
   creating: input/
  inflating: input/get_list.py.ipynb  
  inflating: input/.DS_Store         
  inflating: __MACOSX/input/._.DS_Store  
  inflating: input/natural.txt       
   creating: input/input/
   creating: input/.ipynb_checkpoints/
  inflating: input/input/348.png     
  inflating: input/input/412.png     
  inflating: input/input/374.png     
  inflating: input/input/360.png     
  inflating: input/input/406.png     
  inflating: input/input/638.png     
  inflating: input/input/176.png     
  inflating: input/input/610.png     
  inflating: input/input/604.png     
  inflating: input/input/162.png     

In [ ]:
!rm -rf Transweather
!cp /content/drive/MyDrive/derain/Transweather_orig.zip .
!unzip Transweather_orig.zip
!python testme.py -exp_name=./Transweather

Archive:  Transweather_orig.zip
   creating: Transweather/
  inflating: Transweather/latest     
  inflating: Transweather/best       
Seed:	19
cuda
--- Testing starts! ---
348.png
412.png
374.png
360.png
406.png
638.png
176.png
610.png
604.png
162.png
002.png
016.png
228.png
566.png
200.png
214.png
572.png
599.png
215.png
573.png
567.png
201.png
229.png
017.png
003.png
605.png
163.png
177.png
611.png
639.png
407.png
413.png
375.png
349.png
363.png
377.png
411.png
149.png
161.png
607.png
613.png
175.png
015.png
001.png
559.png
571.png
203.png
565.png
202.png
564.png
570.png
216.png
558.png
014.png
028.png
612.png
174.png
160.png
606.png
389.png
376.png
410.png
404.png
362.png
366.png
400.png
414.png
372.png
399.png
602.png
170.png
616.png
158.png
010.png
004.png
038.png
212.png
574.png
560.png
206.png
561.png
207.png
213.png
575.png
039.png
005.png
011.png
159.png
171.png
617.png
603.png
165.png
398.png
415.png
367.png
401.png
371.png
365.png
359.png
615.png
167.png
601.png
629.png
198

In [ ]:
!mkdir /drive/data
!cp -r data/test/input/input /drive/data
!cp -r ./results/natural/Transweather /drive/data/deweather 

In [ ]:
%cd ..

# Run retinexnet 

In [ ]:
!git clone https://github.com/aasharma90/RetinexNet_PyTorch.git
%cd RetinexNet_PyTorch/

In [ ]:
!mkdir -p /drive/data/retinexnet
!python predict.py --data_dir /drive/data/deweather --res_dir /drive/data/retinexnet

# Run median + sharpening

In [14]:
%cd ..

/content


In [ ]:
import cv2
import os
from glob import glob 
from tqdm.notebook import tqdm

In [19]:
!cp -r TransMod/data/test/input/input /drive/data

In [22]:
!mkdir -p /drive/data/postprocessed

In [ ]:
def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.0, threshold=0):
    """Return a sharpened version of the image, using an unsharp mask."""
    # https://en.wikipedia.org/wiki/Unsharp_masking
  
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

In [ ]:
cv2.media

In [ ]:
out_dir = '/drive/data/postprocessed'
for fname in tqdm('/drive/data/retinexnet/*.png'):
  imname = os.path.basename(fname)
  image = cv2.imread(fname)
  image = unsharp_mask(image)

In [ ]:
print('help')

# Join for the presentation

In [ ]:
os.listdir